In [190]:
import pandas as pd
import numpy as np
import os
import cmath 
import skrf as rf
import matplotlib.pyplot as plt
from tkinter import *
from tkinter import ttk
from numpy.linalg import multi_dot

In [191]:
def parse_s2p(file):
    data_s2p = pd.read_table(file, sep=" ", header=None,  names=["Frequency ", "S11 real", "S11 imaginary", "S12 real", "S12 imaginary",  "S21 real", "S21 imaginary",  "S22 real", "S22 imaginary" ])
    data_s2p.drop(data_s2p.head(3).index,inplace=True)
    return data_s2p

In [192]:
def to_mag_phase(s2p_dataframe):
    s2p_dataframe_array = (np.asarray(s2p_dataframe)).astype('float32')
#     s2p_dataframe_array
    magS11 = []  
    angS11 = [] 
    magS21 = []  
    angS21 = []  
    magS12 = [] 
    angS12 = [] 
    magS22 = [] 
    angS22 = []
    
    for i in range(len(s2p_dataframe_array)): 
        magS11.append(rf.mathFunctions.complex_2_magnitude(complex(s2p_dataframe_array[i][1], s2p_dataframe_array[i][2])))
        angS11.append(rf.mathFunctions.complex_2_degree(complex(s2p_dataframe_array[i][1], s2p_dataframe_array[i][2])))
        magS12.append(rf.mathFunctions.complex_2_magnitude(complex(s2p_dataframe_array[i][3], s2p_dataframe_array[i][4])))
        angS12.append(rf.mathFunctions.complex_2_degree(complex(s2p_dataframe_array[i][3], s2p_dataframe_array[i][4])))
        magS21.append(rf.mathFunctions.complex_2_magnitude(complex(s2p_dataframe_array[i][5], s2p_dataframe_array[i][6])))
        angS21.append(rf.mathFunctions.complex_2_degree(complex(s2p_dataframe_array[i][5], s2p_dataframe_array[i][6])))
        magS22.append(rf.mathFunctions.complex_2_magnitude(complex(s2p_dataframe_array[i][7], s2p_dataframe_array[i][8])))
        angS22.append(rf.mathFunctions.complex_2_degree(complex(s2p_dataframe_array[i][7], s2p_dataframe_array[i][8])))
        
    freq = s2p_dataframe_array[:, 0]
    new_file = []
    
    with open(s2p_file) as f:
        f = f.readlines()
        new_file.append(f[0])
        new_file.append(f[1])
        new_file.append(f[2])
        for i in range(len(s2p_dataframe_array)):
            new_file.append(str('{:.2e}'.format(freq[i])) + ' ' + str('{:.2e}'.format(magS11[i])) + ' ' + str('{:.2e}'.format(angS11[i])) + 
                                           ' ' + str('{:.2e}'.format(magS12[i])) + ' ' + str('{:.2e}'.format(angS12[i])) +
                                           ' ' + str('{:.2e}'.format(magS21[i])) + ' ' + str('{:.2e}'.format(angS21[i])) +
                                           ' ' + str('{:.2e}'.format(magS22[i])) + ' ' + str('{:.2e}'.format(angS22[i]))
                           )
    with open('C:/Users/Dell/Desktop/EE work/s2p/SEI_VGQ0VDQ000_T25_s2p/AA1_updated.s2p', 'w') as of:
        of.write('\n'.join(new_file))

In [187]:
s2p_file = 'C:/Users/Dell/Desktop/EE work/s2p/SEI_VGQ0VDQ000_T25_s2p/AA1.s2p'
dir_name = os.path.dirname(s2p_file)
s2p_dataframe = parse_s2p(s2p_file)
to_mag_phase(s2p_dataframe)
# s2p_dataframe

In [193]:
new_s2p_file = 'C:/Users/Dell/Desktop/EE work/s2p/SEI_VGQ0VDQ000_T25_s2p/AA1_updated.s2p'
ntwk = rf.Network(new_file)
s2p_file_t = ntwk.t
# new_file_t

In [194]:
manifold_in = 'Downloads/UnitCell_Manifold_in_wLauncher.s2p'
manifold_ntwk_in = rf.Network(manifold_in)
manifold_ntwk_in_t = manifold_ntwk_in.t
# manifold_ntwk_in_t

In [195]:
manifold_out = 'Downloads/UnitCell_Manifold_out_wLauncher.s2p'
manifold_ntwk_out = rf.Network(manifold_out)
manifold_ntwk_out_t = manifold_ntwk_out.t
# manifold_ntwk_out_t

In [196]:
def get_T(file):
    T11 = []
    T12 = []
    T21 = []
    T22 = []
    for i in range(len(file)):
        T11.append(file[i][0][0])
        T12.append(file[i][0][1])
        T21.append(file[i][1][0])
        T22.append(file[i][1][1])
    
    return T11, T12, T21, T22

In [197]:
s2p_T11, s2p_T12, s2p_T21, s2p_T22 = get_T(new_file_t)
manifold_ntwk_in_T11, manifold_ntwk_in_T12, manifold_ntwk_in_T21, manifold_ntwk_in_T22 = get_T(manifold_ntwk_in_t)
manifold_ntwk_out_T11, manifold_ntwk_out_T12, manifold_ntwk_out_T21, manifold_ntwk_out_T22 = get_T(manifold_ntwk_out_t)

In [198]:
s2p = [s2p_T11, s2p_T12, s2p_T21, s2p_T22]
manifold_in = [manifold_ntwk_in_T11, manifold_ntwk_in_T12, manifold_ntwk_in_T21, manifold_ntwk_in_T22]
manifold_out = [manifold_ntwk_out_T11, manifold_ntwk_out_T12, manifold_ntwk_out_T21, manifold_ntwk_out_T22]


In [199]:
def matrix(lists, i):
    mat = np.zeros((2,2), dtype = "complex")
    mat[0][0], mat[0][1] = lists[i][0][0], lists[i][0][1]
    mat[1][0], mat[1][1] = lists[i][1][0], lists[i][1][1]    
    return mat

In [200]:
def inv_matrix(lists, i):
    mat = matrix(lists, i)
    return np.linalg.inv(mat)

In [201]:
def mat_calculation(manifold_ntwk_in_t, s2p_file_t, manifold_ntwk_out_t):
    
    de_embedded_t = []
    for i in range(len(s2p_file_t)):
        manifold_in_matrix = inv_matrix(manifold_ntwk_in_t, i)
        s2p_matrix = matrix(s2p_file_t, i)
        manifold_out_matrix_ = inv_matrix(manifold_ntwk_out_t, i)
        
        result_mat = multi_dot([manifold_in_matrix, s2p_matrix, manifold_out_matrix])
        
        de_embedded_t.append(result_mat)

    return de_embedded_t

In [202]:
de_embedded_t = mat_calculation(manifold_ntwk_in_t, s2p_file_t, manifold_ntwk_out_t)

In [207]:
len(de_embedded_t)

398

In [177]:
# s2p_matrix_1 = matrix_1(new_file_t, 0)
# s2p_matrix_1

array([[ 6.19381092e-01+6.81985336e+01j, -2.25154758e-01-1.42061627e-02j],
       [ 2.86692958e-01+1.43909929e-02j,  1.97564279e-05-1.39859861e-02j]])

In [179]:
# manifold_in_matrix_1 = inv_matrix_1(manifold_ntwk_in_t, 0)
# manifold_in_matrix_1

array([[ 1.00137221+0.00227213j, -0.00139202+0.00047426j],
       [ 0.00136661-0.00048154j,  0.99862286-0.00226457j]])

In [181]:
# manifold_out_matrix_1 = inv_matrix_1(manifold_ntwk_out_t, 0)
# manifold_out_matrix_1

array([[ 1.00021949e+00+0.00235574j, -2.27148544e-04+0.00066711j],
       [ 2.20968171e-04-0.00063597j,  9.99775388e-01-0.0023544j ]])

In [183]:
# from numpy.linalg import multi_dot
# result_mat_1 = multi_dot([manifold_in_matrix_1, s2p_matrix_1, manifold_out_matrix_1])
# result_mat_1

array([[ 0.30403011+6.83098646e+01j, -0.27107388-2.93863934e-02j],
       [ 0.31982736+1.07398898e-01j, -0.0005029 -1.36846112e-02j]])